<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/similar_search_models_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install scikit-learn --quiet
!pip install faiss-gpu transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 48.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
import re

In [3]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from tensorflow.keras.models import load_model

# model_v3
# model_v4
# model_freeze_classifer

custom_objects = {'TFBertModel': TFBertModel}
model_v3data = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert.h5', custom_objects=custom_objects)
model_v4data = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_v4.h5', custom_objects=custom_objects)
model_freeze_classifer = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_freezeclassifer.h5', custom_objects=custom_objects)

In [5]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
model_freeze_classifer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [6]:
# cls extraction
def generate_embedding_cls(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)


In [12]:
# maximum pooling

def generate_embedding_max(text, model):
    # same input
    input_ids = model.get_layer('input_ids').input
    attention_mask = model.get_layer('attention_mask').input
    token_type_ids = model.get_layer('token_type_ids').input
    #get bert output
    bert_layer = model.get_layer('tf_bert_model_2')
    bert_output = bert_layer([input_ids, attention_mask, token_type_ids])
    # Apply max pooling on the sequence output (last_hidden_state: (batch, sequence, embedding dimensions))
    pooled_output = GlobalMaxPooling1D()(bert_output[0])
    # Create a new model for embedding extraction
    embedding_model = Model(inputs=[input_ids, attention_mask, token_type_ids],
                            outputs=pooled_output)

    bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
    bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]

    return embedding_model.predict(bert_train_inputs)

In [13]:
#check the size of embeddings
text = """As Jose, a student at Johnnyfurt Higher Institute, I have learned the importance of time management and organization through my academic journey. With my student ID number DNM7052, I have had the opportunity to engage in various lI am doing H1B SPONSOR FOR L1/L2/OPT at US, NY, New York  """
embeddings_v3 = generate_embedding_cls(text, model_v3data)
embeddings_v4 = generate_embedding_cls(text, model_v4data)
embeddings_freeze_classifer = generate_embedding_cls(text, model_freeze_classifer)
embeddings_freeze_classifer_max = generate_embedding_max(text, model_freeze_classifer)

print(embeddings_v3.shape, embeddings_v4.shape, embeddings_freeze_classifer.shape, embeddings_freeze_classifer_max.shape)

(1, 768) (1, 768) (1, 768) (1, 768)


In [14]:
# (768,1) to 768
def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings

In [19]:
# create vector databases
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings

index_cls_v3 = faiss.IndexFlatL2(dimension)
index_cls_v4 = faiss.IndexFlatL2(dimension)
index_cls_freeze_classifier = faiss.IndexFlatL2(dimension)
index_max = faiss.IndexFlatL2(dimension)

In [20]:
# embedding: model_v3 + cls
# embedding: model_v4 + cls
# embedding: model_freeze_classifer + cls
# embedding: model_freeze_classifer + max pooling


level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]

for input in level_four_job:
  embeddings = generate_embedding_cls(input, model_v3data)
  index_cls_v3.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_v4data)
  index_cls_v4.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_freeze_classifer)
  index_cls_freeze_classifier.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_max(input, model_freeze_classifer)
  index_max.add(embedding_vectordatabase(embeddings))

print(index_cls_v3.ntotal, index_cls_v4.ntotal, index_cls_freeze_classifier.ntotal, index_max.ntotal)

10 10 10 10


In [32]:
# compare distance for a input and four vector database.
# based on "Relocation benefits for Apple Cupertino campus positions", ask GPT to generate a prompt (#1)
text = """As Taylor Morgan, I am preparing to discuss my health concerns and lifestyle changes due to my recent relocation for a position at Apple's Cupertino campus.
"""
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[462.62048 557.87604 562.0688 ]]
# Indices of closest embeddings for v4 model: [[8 7 3]]
# Distances for v4 model: [[834.939   842.25964 842.5376 ]]
# Indices of closest embeddings for fc model: [[0 3 8]]
# Distances for fc model: [[664.42206 664.51105 665.29034]]
# Indices of closest embeddings for fcm model: [[1 4 6]]
# Distances for fcm model: [[226.28574 239.99889 259.83896]]

#based on "Global travel opportunities for iPhone supplier audits" (#7)
text = """
As Alex Rivera, I am reporting on the health implications and challenges faced due to my role involving global travel for iPhone supplier audits.
"""
# Indices of closest embeddings for v3 model: [[1 7 6]]
# Distances for v3 model: [[175.7427  207.36827 233.06667]]
# Indices of closest embeddings for v4 model: [[7 8 2]]
# Distances for v4 model: [[621.7367  626.84393 627.2821 ]]
# Indices of closest embeddings for fc model: [[4 2 8]]
# Distances for fc model: [[1.6468006 1.7377512 1.7633522]]
# Indices of closest embeddings for fcm model: [[4 8 6]]
# Distances for fcm model: [[52.757095 58.261253 59.005882]]

# 8. longer?
text = """
As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team. Since the implementation of flexible working hours, I've noticed significant changes in my daily routine, some beneficial and others challenging. The ability to adjust my work hours has afforded me a greater work-life balance, allowing for more time spent with family and engaging in personal hobbies. However, without the structure of fixed working hours, I've found it difficult to establish a consistent sleep schedule, leading to occasional insomnia and varying energy levels throughout the day.
"""
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[442.27026 531.46814 534.8041 ]]
# Indices of closest embeddings for v4 model: [[8 6 3]]
# Distances for v4 model: [[849.8129  857.4177  858.35767]]
# Indices of closest embeddings for fc model: [[1 7 8]]
# Distances for fc model: [[156.9984  157.8388  158.15071]]
# Indices of closest embeddings for fcm model: [[1 4 6]]
# Distances for fcm model: [[230.73032 235.79501 268.56027]]

# 8. sentence level
text = """
As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team.
"""
# Indices of closest embeddings for v3 model: [[7 4 1]]
# Distances for v3 model: [[141.25726 142.04306 148.1364 ]]
# Indices of closest embeddings for v4 model: [[5 9 8]]
# Distances for v4 model: [[434.44342 434.8328  437.09268]]
# Indices of closest embeddings for fc model: [[4 8 2]]
# Distances for fc model: [[2.7517064 2.8037987 2.904843 ]]
# Indices of closest embeddings for fcm model: [[1 4 3]]
# Distances for fcm model: [[86.91295  87.148895 97.19384 ]]

# diagnosis, no job related information
text = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[765.25256 848.5805  867.85834]]
# Indices of closest embeddings for v4 model: [[8 7 3]]
# Distances for v4 model: [[829.60986 832.6284  833.5238 ]]
# Indices of closest embeddings for fc model: [[0 3 7]]
# Distances for fc model: [[1051.2994 1052.177  1052.7937]]
# Indices of closest embeddings for fcm model: [[1 4 6]]
# Distances for fcm model: [[419.391   425.74582 439.39606]]

embeddings_v3 = generate_embedding_cls(text, model_v3data)
embeddings_v3 = embedding_vectordatabase(embeddings_v3)

embeddings_v4 = generate_embedding_cls(text, model_v4data)
embeddings_v4 = embedding_vectordatabase(embeddings_v4)

embeddings_freeze_classifer = generate_embedding_cls(text, model_freeze_classifer)
embeddings_freeze_classifer = embedding_vectordatabase(embeddings_freeze_classifer)

embeddings_freeze_classifer_max = generate_embedding_max(text, model_freeze_classifer)
embeddings_freeze_classifer_max = embedding_vectordatabase(embeddings_freeze_classifer_max)

# Search the index
k = 3
distances_v3, indices_v3 = index_cls_v3.search(embeddings_v3, k)
distances_v4, indices_v4 = index_cls_v4.search(embeddings_v4, k)
distances_fc, indices_fc = index_cls_freeze_classifier.search(embeddings_freeze_classifer, k)
distances_fcm, indices_fcm = index_max.search(embeddings_freeze_classifer_max, k)

print("Indices of closest embeddings for v3 model:", indices_v3)
print("Distances for v3 model:", distances_v3)

print("Indices of closest embeddings for v4 model:", indices_v4)
print("Distances for v4 model:", distances_v4)

print("Indices of closest embeddings for fc model:", indices_fc)
print("Distances for fc model:", distances_fc)

print("Indices of closest embeddings for fcm model:", indices_fcm)
print("Distances for fcm model:", distances_fcm)


Indices of closest embeddings for v3 model: [[1 6 7]]
Distances for v3 model: [[765.25256 848.5805  867.85834]]
Indices of closest embeddings for v4 model: [[8 7 3]]
Distances for v4 model: [[829.60986 832.6284  833.5238 ]]
Indices of closest embeddings for fc model: [[0 3 7]]
Distances for fc model: [[1051.2994 1052.177  1052.7937]]
Indices of closest embeddings for fcm model: [[1 4 6]]
Distances for fcm model: [[419.391   425.74582 439.39606]]


In [ ]:
# split the prompt
# a clear blacklist, be specific
# setence level seems off, but if better than paragraph..
# yes/no OK
# accurate? doubtful -> try split the prompt and clear blacklist